In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from models import CNN_Gender, CNN_Age
import cv2
import requests
import json

In [4]:
image_directory = "../data/"

In [5]:
#image size
#200x200x3

In [6]:
ages = []
gender = []
file_name = []
data = []
for file in os.listdir(image_directory):
    split_file = file.split("_")
    if(len(split_file) == 4):
        ages.append(int(split_file[0]))
        gender.append(split_file[1])
        file_name.append(image_directory+file)
        #data.append(plt.imread(image_directory+file))
#data = np.array(data).astype('float16')
#gender = np.array(gender)
#ages = np.array(ages)
labels_dataframe = pd.DataFrame({"ages":ages,"gender":gender,"file_name":file_name})

In [7]:
labels_dataframe

,ages,gender,file_name
0,100,0,../data/100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,../data/100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,../data/100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,../data/100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,../data/100_1_0_20170112213303693.jpg.chip.jpg
...,...,...,...
23700,9,1,../data/9_1_3_20161220222856346.jpg.chip.jpg
23701,9,1,../data/9_1_3_20170104222949455.jpg.chip.jpg
23702,9,1,../data/9_1_4_20170103200637399.jpg.chip.jpg
23703,9,1,../data/9_1_4_20170103200814791.jpg.chip.jpg


In [8]:
def preprocess(image):
    image = image.astype('float16')
    image = image/255
    return image

In [9]:
def create_generator(class_mode,y_col):
    generator = ImageDataGenerator(validation_split = 0.2, preprocessing_function=preprocess)

    train_generator = generator.flow_from_dataframe(

        dataframe = labels_dataframe,
        x_col = 'file_name',
        y_col = y_col,
        subset = "training",
        batch_size = 32,
        shuffle = True,
        class_mode = class_mode,
        target_size = (200,200)

    )

    validation_generator = generator.flow_from_dataframe(
        dataframe = labels_dataframe,
        x_col = 'file_name',
        y_col = y_col,
        subset = "validation",
        batch_size = 32,
        shuffle = True,
        class_mode = class_mode,
        target_size = (200,200) 
    )
    return train_generator,validation_generator

In [ ]:
train_generator, validation_generator = create_generator('binary','gender')
gender_model = CNN_Gender()
gender_model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ['accuracy'])
gender_model.fit_generator(generator=train_generator,validation_data=validation_generator, epochs = 2)

In [47]:
gender_model.save("gender_model")

INFO:tensorflow:Assets written to: gender_model\assets


In [ ]:
train_generator, validation_generator = create_generator('raw','ages')
age_model = CNN_Age()
age_model.compile(optimizer="adam", loss = "mse", metrics = ['mean_squared_error'])
age_model.fit_generator(generator=train_generator,validation_data=validation_generator, epochs = 2)

In [15]:
age_model.save("age_model")

INFO:tensorflow:Assets written to: age_model\assets
